In [3]:
import secrets
import math
from phe import paillier
from app.ecdsa import *

def public_broadcast(message):
    print(f"[BROADCAST] {message}")
    
    
def send_to_peer(i, data):
    print(f"[TO P{i}] {data}")
    
    
    
def shamir_split(secret, t, n, prime):
    coeffs = [secret] + [secrets.randbelow(prime) for _ in range(t-1)]
    shares = []
    for i in range(1, n+1):
        x = i
        # eval polinomio
        fx = 0
        power = 1
        for c in coeffs:
            fx = (fx + c * power) % prime
            power = (power * x) % prime
        shares.append((i, fx))
    return shares

def shamir_combine(shares, prime):
    """
    Interpola polinomio en x=0
    shares = [(x1, y1), (x2, y2), ..., (xk, yk)].
    Retorna f(0).
    """
    total = 0
    k = len(shares)
    for i in range(k):
        xi, yi = shares[i]
        num = 1
        den = 1
        for j in range(k):
            if i != j:
                xj, _ = shares[j]
                num = (num * (-xj)) % prime
                den = (den * (xi - xj)) % prime
        # inverso de den
        inv_den = pow(den, prime-2, prime)
        li = (num * inv_den) % prime
        contribution = (yi * li) % prime
        total = (total + contribution) % prime
    return total



class TrustedDealerCoordinator:
    def __init__(self, n, t, prime_for_shamir):
        self.n = n
        self.t = t
        self.prime = prime_for_shamir
        print("[Coordinator] Generando Paillier KeyPair...")
        self.pubkey, self.privkey = paillier.generate_paillier_keypair()
        self.ciphertext_sum = None
        self.secreto = None
        
    def get_public_key(self):
        return self.pubkey
    
    def receive_final_ciphertext_sum(self, ciph_sum):
        self.ciphertext_sum = ciph_sum
        print("[Coordinator] He recibido la suma cifrada de los x_i.")
    
    def decrypt_and_distribute_shares(self):
        if self.ciphertext_sum is None:
            print("[Coordinator] No tengo nada que desencriptar.")
            return
        x_global = self.privkey.decrypt(self.ciphertext_sum)
        print(f"[Coordinator] x_global = {x_global} (clave privada global en claro)")
        self.secreto = x_global
        
        shares = shamir_split(x_global, self.t, self.n, self.prime)
        for i, (idx, fx) in enumerate(shares, start=1):
            send_to_peer(i, f"Tu share = {fx}")
            participants[i-1].receive_shamir_share(fx)
            
    def collect_shares_and_sign(self, z, chosen_participants):

        shamir_shares = []
        k_total = 0

        for p in chosen_participants:
            # p.pid => x-coordinate, p.share => y-value
            i = p.pid
            f_i = p.share
            # Cada participante también tiene su parte de k => p.k_i
            k_total = (k_total + p.k_i) % S256Point.N

        # 2. Interpolar => x = f(0)
        x_reconstructed = shamir_combine(chosen, S256Point.N)  
        print(x_reconstructed)
        # OJO: si quieres usar 'prime_for_shamir' en vez del orden S256Point.N,
        # asegúrate de la consistencia. A menudo se trabaja mod N.

        # 3. Calcular r = (k_total * G).x
        G = S256Point(
            0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
            0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
        )
        R = k_total * G
        r = R.x.num 

        # 4. inverso de k_total
        k_inv = pow(k_total, S256Point.N - 2, S256Point.N)

        # 5. s = (z + r*x) * k_inv mod N
        s = ((z + r * x_reconstructed) % S256Point.N) * k_inv % S256Point.N

        print(f"[Coordinator] RECONSTRUYÓ x={x_reconstructed} y firmó => r={r}, s={s}")
        return (r, s)
 
            

class Participant:
    def __init__(self, pid, paillier_pubkey):
        self.pid = pid
        self.pk = paillier_pubkey
        self.x_i = None  # se elige en create_secret
        self.share = None  # se recibe luego del Coordinador

    def create_secret(self):
        self.x_i = secrets.randbits(128)  # EJ: 128 bits, a modo de ejemplo
        print(f"[P{self.pid}] Mi secreto local x_i = {self.x_i}")
        return self.x_i

    def encrypt_secret(self):
        if self.x_i is None:
            raise ValueError("No secret to encrypt.")
        ciph = self.pk.encrypt(self.x_i)
        return ciph

    def receive_shamir_share(self, s):
        self.share = s
        print(f"[P{self.pid}] Recibí mi share de la clave global: {s}")
    
    def generate_nonce_share(self):
        """
        Genera su 'k_i' local (parte del nonce).
        """
        self.k_i = secrets.randbelow(S256Point.N - 1) + 1
        print(f"[P{self.pid}] generó k_i = {self.k_i}")
        return self.k_i


In [4]:
prime_for_shamir = S256Point.N
n=5
t=3

coordinator = TrustedDealerCoordinator(n, t, prime_for_shamir)

# 2) Publicamos la clave pública Paillier
pk = coordinator.get_public_key()
public_broadcast(f"PaillierPublicKey => (n={pk.n}, g={pk.g})")

# 3) Participantes
participants = []
for i in range(1, n+1):
    p = Participant(i, pk)
    participants.append(p)

print("\n--- Paso A: cada participante elige su x_i y lo cifra ---")
ciphertexts = []
for p in participants:
    p.create_secret()
    ciph = p.encrypt_secret()
    ciphertexts.append(ciph)
    public_broadcast(f"[P{p.pid}] Ciphertext = {ciph.ciphertext()}")

print("\n--- Paso B: Se suman homomórficamente los ciphertexts ---")

if ciphertexts:
    total_enc = ciphertexts[0]
    for c in ciphertexts[1:]:
        total_enc = total_enc + c  # homomorfía aditiva
else:
    total_enc = None

print("[AGREGADOR] Suma homomórfica co    mpletada. Se envía al Coordinador.")
coordinator.receive_final_ciphertext_sum(total_enc)

print("\n--- Paso C: El Coordinador desencripta y reparte con Shamir ---")
coordinator.decrypt_and_distribute_shares()

print("\n--- Paso D: Simulamos que cada participante recibe su share ---")
for p in participants:
    p.receive_shamir_share(p.share)


[Coordinator] Generando Paillier KeyPair...
[BROADCAST] PaillierPublicKey => (n=32669089560786834202035939294209713647069806616410248392548042851969453169944632240418247174057418711907295804387936885762131170222671985670239333797524992196744301441910423564372015587799991382132540862713490571017131738049181350371677832198916375649767278376678410798618781466174332658408483331773566596597905639950106747145234693195062634027712019022401064910408752801480577307931950493595932987129746980345691454623887931213249833820348585637277511035132198702514779741779411513612261768622349559592494393401058190255951340621447923371631723136464344846949541656666626635697221414010562735009744663385795051084909969030860700357119323609635546628629018139531036862053739381035361646483440007968000197053836768806430820863237705407831553703716661035902109729832123605044365304554193332904526050266611304725705953725675420395863299429134780605419266682025637633406681364170175550726870934456525425199597132358476575615

In [5]:
chosen = [(1, participants[0].share),
           (3, participants[2].share),
           (5, participants[4].share)]
print(f"Elegimos shares => {chosen}")

chosen
recovered_x = shamir_combine(chosen, prime_for_shamir)
recovered_x == coordinator.secreto
recovered_x

Elegimos shares => [(1, 3707812106665693900730894858939871017766236333010793249844370491068014820508), (3, 46731156086705106589999021238757211947325342209305777933871523587653881674056), (5, 21439370518372357039437944825803443551624769465082399700237220252831913879403)]


828390516822003957184683959066453961035

In [6]:


e = recovered_x
pk_e = PublicKey(e)
print(e)
print(pk_e.point)


828390516822003957184683959066453961035
S256Point(048fa64f959b80f44435ae783ba640662c1ec98631a20569f512876a5347628d, 3ef2a92700c2ad4c8fe8c18f6258285aef6e06bd1bebba7e2c25676c68cf5f30)


In [7]:
chosen_participants = [participants[0], participants[2], participants[4]]
participants

for cp in chosen_participants:
    cp.generate_nonce_share()
    
z = int.from_bytes(hash256(b'my message'), 'big') #HASH


r,s = coordinator.collect_shares_and_sign(z, chosen_participants)


[P1] generó k_i = 25164844316778384377199263277847593836694742641701226851389119876993606169322
[P3] generó k_i = 101752452358044185297969566124579231825603487108087056547187610511808255369851
[P5] generó k_i = 18031387746215856543467114647703160170449232891690220966597282771855174091830
828390516822003957184683959066453961035
[Coordinator] RECONSTRUYÓ x=828390516822003957184683959066453961035 y firmó => r=87376378296017076441039352887037296007977074048766870832347927630535489835665, s=96626326737240062039250454089744884093553036372512115907418260120991817909161


In [8]:
print(r,s)

s

87376378296017076441039352887037296007977074048766870832347927630535489835665 96626326737240062039250454089744884093553036372512115907418260120991817909161


96626326737240062039250454089744884093553036372512115907418260120991817909161

In [9]:
G = S256Point(0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)
s_inv = pow(s, S256Point.N - 2, S256Point.N)
u = z * s_inv % S256Point.N
v = r * s_inv % S256Point.N
total = u * G + v * pk_e.point
total.x.num == r

True